# State Reducers - Practice Exercises

## Overview
This notebook provides hands-on exercises to practice working with state reducers in LangGraph. You'll learn how to customize how state updates are applied using different reducer functions.

## Learning Objectives
By the end of these exercises, you will:
- Understand the default state update behavior (overwriting)
- Know how to use built-in reducers like `add` for lists
- Create custom reducers for complex state update logic
- Handle concurrent state updates from multiple nodes
- Apply reducers to real-world scenarios like accumulating data

## Prerequisites
- Completed the state-reducers.ipynb tutorial
- Understanding of state schema concepts

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph

## Exercise 1: Understanding Default Overwriting Behavior

### Task
Create a simple counter system that demonstrates the default overwriting behavior of LangGraph state updates.

### TODO: Implement a basic counter with overwriting behavior

In [ ]:
from typing_extensions import TypedDict
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# TODO: Define a simple state with a counter
class CounterState(TypedDict):
    # TODO: Add counter field as int
    pass

# TODO: Implement nodes that increment the counter
def increment_by_one(state):
    print(f"Current counter: {state['counter']}, incrementing by 1")
    # TODO: Return new counter value (current + 1)
    pass

def increment_by_five(state):
    print(f"Current counter: {state['counter']}, incrementing by 5")
    # TODO: Return new counter value (current + 5)
    pass

def increment_by_ten(state):
    print(f"Current counter: {state['counter']}, incrementing by 10")
    # TODO: Return new counter value (current + 10)
    pass

In [ ]:
# TODO: Build a sequential graph
builder = StateGraph(CounterState)
# TODO: Add nodes
# TODO: Add edges to create: START -> increment_by_one -> increment_by_five -> increment_by_ten -> END

graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test the sequential execution
result = graph.invoke({"counter": 0})
print(f"Final counter value: {result['counter']}")
# Expected: 16 (0 + 1 + 5 + 10)

### TODO: Now test with branching (parallel execution)

What happens when multiple nodes run in parallel and update the same state key?

In [ ]:
# TODO: Build a branching graph where all increment nodes run in parallel
builder_parallel = StateGraph(CounterState)
# TODO: Add all increment nodes
# TODO: Add edges so all three nodes run in parallel from START
# All nodes should connect to a final "collect" node

def collect_results(state):
    print(f"Final collection - counter value: {state['counter']}")
    return {}

# TODO: Add collect_results node and connect parallel nodes to it

graph_parallel = builder_parallel.compile()
display(Image(graph_parallel.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test parallel execution
result_parallel = graph_parallel.invoke({"counter": 0})
print(f"Final counter value with parallel execution: {result_parallel['counter']}")
# Question: Is this the result you expected? Why or why not?

## Exercise 2: Using Built-in Reducers

### Task
Create a shopping cart system that uses the built-in `add` reducer to accumulate items.

### TODO: Define state with list reducer

In [ ]:
from typing import List, Annotated
from langgraph.graph import add

# TODO: Define shopping cart state with list reducer
class ShoppingCartState(TypedDict):
    # TODO: Use Annotated to specify that items should use the add reducer
    # items: Annotated[List[str], add]
    total_price: float

In [ ]:
# TODO: Implement nodes that add items to cart
def add_electronics(state):
    print("Adding electronics to cart")
    # TODO: Return dict with items list containing electronics items
    items = ["Laptop", "Mouse", "Keyboard"]
    # TODO: Also update total_price by adding 1500.0
    pass

def add_books(state):
    print("Adding books to cart")
    # TODO: Return dict with items list containing book items
    items = ["Python Programming", "Data Science Handbook"]
    # TODO: Also update total_price by adding 80.0
    pass

def add_groceries(state):
    print("Adding groceries to cart")
    # TODO: Return dict with items list containing grocery items
    items = ["Apples", "Bread", "Milk"]
    # TODO: Also update total_price by adding 25.0
    pass

In [ ]:
# TODO: Build parallel shopping graph
builder_shopping = StateGraph(ShoppingCartState)
# TODO: Add all shopping nodes to run in parallel
# TODO: Connect to a final checkout node

def checkout(state):
    print(f"Checking out with {len(state['items'])} items")
    print(f"Items: {state['items']}")
    print(f"Total: ${state['total_price']}")
    return {}

graph_shopping = builder_shopping.compile()
display(Image(graph_shopping.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test shopping cart with parallel item addition
result_shopping = graph_shopping.invoke({"items": [], "total_price": 0.0})
print(f"Final cart: {result_shopping}")
# The items should be accumulated, not overwritten!

## Exercise 3: Custom Reducers

### Task
Create a data analysis system with custom reducers that accumulate statistics from multiple data processing nodes.

### TODO: Define custom reducer functions

In [ ]:
from typing import Dict, Any

# TODO: Implement custom reducer for merging dictionaries
def merge_stats(current: Dict[str, Any], update: Dict[str, Any]) -> Dict[str, Any]:
    """Custom reducer that merges statistics dictionaries."""
    if current is None:
        return update
    
    merged = current.copy()
    # TODO: Implement logic to merge statistics
    # For numeric values, add them together
    # For lists, concatenate them
    # For strings, keep the update value
    pass

# TODO: Implement custom reducer for accumulating maximum values
def max_reducer(current: float, update: float) -> float:
    """Custom reducer that keeps the maximum value."""
    # TODO: Return the maximum of current and update
    pass

# TODO: Implement custom reducer for accumulating minimum values  
def min_reducer(current: float, update: float) -> float:
    """Custom reducer that keeps the minimum value."""
    # TODO: Return the minimum of current and update (handle None case)
    pass

In [ ]:
# TODO: Define analytics state with custom reducers
class AnalyticsState(TypedDict):
    # TODO: Use Annotated with your custom reducers
    # stats: Annotated[Dict[str, Any], merge_stats]
    # max_value: Annotated[float, max_reducer]  
    # min_value: Annotated[float, min_reducer]
    dataset_name: str

In [ ]:
# TODO: Implement data processing nodes
def process_user_data(state):
    print("Processing user data...")
    # TODO: Return stats dict with user data statistics
    user_stats = {
        "user_count": 1000,
        "active_users": 750,
        "categories": ["premium", "basic"]
    }
    # TODO: Also return max_value and min_value
    pass

def process_sales_data(state):
    print("Processing sales data...")
    # TODO: Return stats dict with sales data statistics
    sales_stats = {
        "total_sales": 50000.0,
        "transaction_count": 500,
        "categories": ["electronics", "books"]
    }
    # TODO: Also return max_value and min_value
    pass

def process_performance_data(state):
    print("Processing performance data...")
    # TODO: Return stats dict with performance data statistics
    perf_stats = {
        "avg_response_time": 150.5,
        "error_count": 5,
        "uptime_percentage": 99.9
    }
    # TODO: Also return max_value and min_value
    pass

In [ ]:
# TODO: Build analytics processing graph
builder_analytics = StateGraph(AnalyticsState)
# TODO: Add all processing nodes to run in parallel
# TODO: Connect to a report generation node

def generate_report(state):
    print("\n=== Analytics Report ===")
    print(f"Dataset: {state['dataset_name']}")
    print(f"Merged Statistics: {state['stats']}")
    print(f"Maximum Value: {state['max_value']}")
    print(f"Minimum Value: {state['min_value']}")
    return {}

graph_analytics = builder_analytics.compile()
display(Image(graph_analytics.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test analytics processing
initial_analytics_state = {
    "stats": {},
    "max_value": float('-inf'),
    "min_value": float('inf'),
    "dataset_name": "Q4 Business Analytics"
}

result_analytics = graph_analytics.invoke(initial_analytics_state)
print(f"\nFinal analytics result: {result_analytics}")

## Exercise 4: Complex State Management with Multiple Reducers

### Task
Create a collaborative document editing system where multiple editors can make changes simultaneously. Use different reducers for different types of updates.

### TODO: Implement document collaboration system

In [ ]:
from typing import Set
from datetime import datetime

# TODO: Define custom reducers for document collaboration
def merge_content(current: str, update: str) -> str:
    """Merge document content updates."""
    # TODO: Implement simple content merging (concatenate with newline)
    pass

def merge_editors(current: Set[str], update: Set[str]) -> Set[str]:
    """Merge sets of editors."""
    # TODO: Implement set union
    pass

def latest_timestamp(current: str, update: str) -> str:
    """Keep the latest timestamp."""
    # TODO: Compare timestamps and return the later one
    pass

# TODO: Define document collaboration state
class DocumentState(TypedDict):
    # TODO: Use Annotated with custom reducers
    # content: Annotated[str, merge_content]
    # editors: Annotated[Set[str], merge_editors]  
    # word_count: Annotated[int, add]
    # last_modified: Annotated[str, latest_timestamp]
    title: str

In [ ]:
# TODO: Implement editor nodes
def alice_edit(state):
    print("Alice is editing the document...")
    # TODO: Return document updates from Alice
    content_addition = "\n\nIntroduction:\nThis document covers the basics of state management."
    # TODO: Add to content, editors set, word_count, and last_modified
    pass

def bob_edit(state):
    print("Bob is editing the document...")
    # TODO: Return document updates from Bob
    content_addition = "\n\nAdvanced Topics:\nWe'll explore complex scenarios and best practices."
    # TODO: Add to content, editors set, word_count, and last_modified
    pass

def carol_edit(state):
    print("Carol is editing the document...")
    # TODO: Return document updates from Carol
    content_addition = "\n\nConclusion:\nState management is crucial for complex applications."
    # TODO: Add to content, editors set, word_count, and last_modified
    pass

In [ ]:
# TODO: Build collaborative editing graph
builder_doc = StateGraph(DocumentState)
# TODO: Add editor nodes to run in parallel
# TODO: Connect to a final review node

def review_document(state):
    print("\n=== Document Review ===")
    print(f"Title: {state['title']}")
    print(f"Editors: {state['editors']}")
    print(f"Word Count: {state['word_count']}")
    print(f"Last Modified: {state['last_modified']}")
    print(f"Content: {state['content']}")
    return {}

graph_doc = builder_doc.compile()
display(Image(graph_doc.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test collaborative document editing
initial_doc_state = {
    "content": "State Management Guide",
    "editors": set(),
    "word_count": 3,
    "last_modified": "2024-01-01T10:00:00",
    "title": "Complete Guide to State Management"
}

result_doc = graph_doc.invoke(initial_doc_state)
print(f"\nFinal document state keys: {result_doc.keys()}")

## Exercise 5: Error Handling in Reducers

### Task
Create robust reducers that handle edge cases and errors gracefully.

### TODO: Implement error-safe reducers

In [ ]:
# TODO: Implement safe numeric reducer
def safe_add_reducer(current: float, update: float) -> float:
    """Safely add numeric values, handling None and type errors."""
    # TODO: Handle None values and type conversion errors
    try:
        # TODO: Convert to float and add
        pass
    except (TypeError, ValueError):
        # TODO: Return appropriate fallback value
        pass

# TODO: Implement safe list merger
def safe_list_merge(current: List[Any], update: List[Any]) -> List[Any]:
    """Safely merge lists, handling None values."""
    # TODO: Handle None values and ensure both inputs are lists
    pass

# TODO: Implement safe dictionary merger
def safe_dict_merge(current: Dict[str, Any], update: Dict[str, Any]) -> Dict[str, Any]:
    """Safely merge dictionaries with conflict resolution."""
    # TODO: Handle None values and key conflicts
    pass

In [ ]:
# TODO: Define state with error-safe reducers
class RobustState(TypedDict):
    # TODO: Use your safe reducers
    # total: Annotated[float, safe_add_reducer]
    # items: Annotated[List[Any], safe_list_merge]
    # metadata: Annotated[Dict[str, Any], safe_dict_merge]
    status: str

In [ ]:
# TODO: Implement nodes that might produce problematic data
def node_with_none_values(state):
    print("Node producing None values...")
    # TODO: Return some None values to test error handling
    pass

def node_with_wrong_types(state):
    print("Node producing wrong types...")
    # TODO: Return wrong types to test error handling
    pass

def node_with_valid_data(state):
    print("Node producing valid data...")
    # TODO: Return valid data
    pass

In [ ]:
# TODO: Build error-testing graph
builder_robust = StateGraph(RobustState)
# TODO: Add all test nodes
# TODO: Connect to validation node

def validate_final_state(state):
    print("\n=== Final State Validation ===")
    print(f"Total: {state['total']} (type: {type(state['total'])})")
    print(f"Items: {state['items']} (type: {type(state['items'])})")
    print(f"Metadata: {state['metadata']} (type: {type(state['metadata'])})")
    print(f"Status: {state['status']}")
    return {}

graph_robust = builder_robust.compile()
display(Image(graph_robust.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test error handling
initial_robust_state = {
    "total": 10.0,
    "items": ["initial_item"],
    "metadata": {"version": 1},
    "status": "processing"
}

try:
    result_robust = graph_robust.invoke(initial_robust_state)
    print("\nGraph executed successfully despite problematic inputs!")
except Exception as e:
    print(f"Graph failed with error: {e}")

## Challenge Exercise: Dynamic Reducer Selection

### Task
Create a system where the reducer behavior can change based on the current state or external conditions. This demonstrates advanced reducer patterns.

### TODO: Implement dynamic reducer system

In [ ]:
# TODO: Implement conditional reducer
def conditional_number_reducer(current: Dict[str, Any], update: Dict[str, Any]) -> Dict[str, Any]:
    """Reducer that changes behavior based on current state."""
    # TODO: Implement logic that:
    # - If current['mode'] == 'add', add the numbers
    # - If current['mode'] == 'multiply', multiply the numbers  
    # - If current['mode'] == 'max', keep the maximum
    pass

# TODO: Define dynamic state
class DynamicState(TypedDict):
    # TODO: Use the conditional reducer
    # data: Annotated[Dict[str, Any], conditional_number_reducer]
    mode: str
    step: int

In [ ]:
# TODO: Implement nodes that change modes
def addition_mode(state):
    print(f"Step {state['step']}: Setting addition mode")
    # TODO: Return mode='add' and some numeric data
    pass

def multiplication_mode(state):
    print(f"Step {state['step']}: Setting multiplication mode")
    # TODO: Return mode='multiply' and some numeric data
    pass

def max_mode(state):
    print(f"Step {state['step']}: Setting max mode")
    # TODO: Return mode='max' and some numeric data
    pass

def increment_step(state):
    return {"step": state['step'] + 1}

## Summary

In these exercises, you've practiced:
- Understanding default overwriting behavior vs. reducer-based accumulation
- Using built-in reducers like `add` for lists
- Creating custom reducers for complex state merging logic
- Handling errors and edge cases in reducers
- Building systems with multiple concurrent state updates

Key takeaways:
- **Default behavior**: LangGraph overwrites state values by default
- **Reducers**: Allow you to customize how state updates are combined
- **Built-in reducers**: `add` for lists, basic arithmetic operations
- **Custom reducers**: Enable complex merging logic for your specific use cases
- **Error handling**: Important for robust production systems
- **Parallel execution**: Reducers are essential when multiple nodes update the same state key

Next, continue with the multiple-schemas exercises to learn about working with different input/output schemas!